In [98]:
pip install groq scikit-learn openai

Note: you may need to restart the kernel to use updated packages.


# Load input file

In [99]:
project_title_map = {
        "archives_space": "Archives Space Project",
        "archives_space_old": "Archives Space Project",
        "neurohub": "NeuroHub Project",
        "open_spending": "Open Spending Project",
        "open_spending_old": "Open Spending Project",
        "planning_poker": "Planning Poker Project",
        "recycling": "Recycling Project",
        "color_ide": "ColorIDE Project"
    }
projects = ["archives_space", "neurohub", "open_spending", "planning_poker", "recycling", "color_ide"]

selected_project = projects[0] # select which project you want to test with

project_path = "./input_files/user_stories_{}.json".format(selected_project)
with open(project_path, 'r') as file:
        project_content = file.read()
        project_title = project_title_map[selected_project]

# Load model and configure model parameters

In [100]:
import groq
from openai import OpenAI

client = OpenAI(api_key="", base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_version = "gemini-2.0-flash"


#client = groq.Client(api_key="")
#model_version = "llama-3.3-70b-versatile"

#client = OpenAI(api_key="")
#model_version = "gpt-4o"

#model_version = "llama3.1:8b-instruct-fp16"

num_ctx = 15000 # context length is higher because of the refinement process
temperature = 0.000000001  # should be kept 0 for deterministic results, default value 0.8
#temperature = 0
top_p = 0.0000001
model_name = model_version

one_shot = False

add_pattern_descriptions = False

# Get first assessment from model

In [101]:

def get_architecture_score(project_title, project_content, architecture_pattern, model_name):
    generator_system_message = {'role': 'system', 'content': """
    You are a software architect. Your job is to get a list of categorized user stories with a description, analyze them in detail and
    assign a score for the given architecture pattern depending on their relevance to the project and 
    if it would prove useful in the implementation of the described project.
    IMPORTANT: You MUST respond with ONLY a single digit from 1 to 5, representing the score. DO NOT provide any other text, explanation, introduction, internal reasoning or formatting.
    IMPORTANT: The user stories are categorized and given in json format, analyze them really carefully before you answer.
    IMPORTANT: Your answer must relate to the suitability of the given software architecture pattern to the described project.

    EXAMPLE 1:

    Project Title: <title>

    User Stories:

    <user stories>

    Architecture Pattern: Layered Architecture

    Score: <int>

    EXAMPLE 2:

    Project Title: <title>

    User Stories:

    <user stories>

    Architecture Pattern: Microservices Architecture

    Score: <int>
    """}
    user_message = {'role': 'user', 'content': f"""
    Project Title: {project_title}

    User Stories:

    {project_content}

    Architecture Pattern: {architecture_pattern}

    Score: 
    """}
    messages = [generator_system_message, user_message]  # Reset messages for each pattern

    response = client.chat.completions.create(model=model_name, messages=messages, temperature=temperature, max_completion_tokens=1)
    message = response.choices[0].message
    print(message.content)

    try:
        score = int(message.content.strip())
        if 1 <= score <= 5:
            return score
        else:
            print(f"Invalid score: {score}. Model returned a number outside the range 1-5.")
            return None  # Or raise an exception
    except ValueError:
        print(f"Invalid response: '{message.content}'. Model did not return a valid integer.")
        return None  # Or raise an exception

In [102]:
architecture_patterns = ["Layered Architecture", "Event-Driven Architecture (Pub-Sub Architecture)",
 "Microkernel Architecture", "Microservices Architecture", "Space-Based Architecture", "Pipeline Architecture (Pipe-Filter Architecture)", "Client-Server Architecture"]

for pattern in architecture_patterns:
    score = get_architecture_score(project_title, project_content, pattern, model_name)
    if score is not None:
        print(f"Score for {pattern}: {score}")

5
Score for Layered Architecture: 5
4
Score for Event-Driven Architecture (Pub-Sub Architecture): 4
4
Score for Microkernel Architecture: 4
4
Score for Microservices Architecture: 4
2
Score for Space-Based Architecture: 2
5
Score for Pipeline Architecture (Pipe-Filter Architecture): 5
5
Score for Client-Server Architecture: 5
